In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd

df=pd.read_csv('train_data.csv')
df.head()

,id,date,prix,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,...,vue_note,etat_note,design_note,annee_construction,annee_renovation,m2_interieur_15voisins,m2_jardin_15voisins,zipcode,lat,long
0,4443800785,2014-11-21T00:00:00Z,481000,2,1.00,150.501672,360.460795,85.470085,65.031587,1.0,...,0,4,7,1924,0,123.560015,360.460795,98117,47.6855,-122.391
1,2722059013,2015-02-04T00:00:00Z,550000,2,1.00,117.985879,4046.822742,117.985879,0.000000,1.0,...,0,4,5,1908,0,173.727239,646.599777,98042,47.3651,-122.165
2,8856004730,2014-09-17T00:00:00Z,199950,2,2.75,147.714604,1943.236715,147.714604,0.000000,1.5,...,0,3,5,1920,0,121.701970,557.413601,98001,47.2786,-122.250
3,2473003210,2015-03-13T00:00:00Z,364808,3,1.75,215.533259,731.605351,150.501672,65.031587,1.0,...,0,3,8,1967,0,184.875511,903.010033,98058,47.4524,-122.146
4,6743700335,2014-06-04T00:00:00Z,470000,3,2.00,167.224080,1176.978818,167.224080,0.000000,1.0,...,0,3,7,1956,1990,183.017466,908.119658,98033,47.6935,-122.173


In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, r2_score


df = pd.read_csv("train_data.csv")  

# Calcul du prix moyen par zipcode
df["prix_moy_zipcode"] = df.groupby("zipcode")["prix"].transform("mean")

# Pour les variables explicatives, j'ai supprimé toutes celles jugées difficiles à traiter 
X = df.drop(columns=["prix","id","date","zipcode","lat",'long'])  # Variables explicatives
y = df["prix"]  # Target

# Train/test avec 20% pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline : peremet d'imputer les valeurs manquantes avec la moyenne, standardise les valeurs (moyenne 0 e-t 1), sélection des 
# variables les plus pertinentes et regression linéaire imposée par le sujet
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),  # Remplace les NaNs par la moyenne
    ("scaler", StandardScaler()),  # Standardisation
    ("feature_selection", SelectKBest(score_func=f_regression, k=5)),  # Sélection de features
    ("model", LinearRegression())  # Régression linéaire
])

# Entraînement 
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Calcul du RMSE comme demandé dans le sujet et r2 car parlant 
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE du modèle : {rmse:.2f}")
print(f"R² du modèle : {r2:.3f}")

# GridSearchCV pour optimiser la sélection de variables et le modèle
param_grid = {
    "feature_selection__k": [3, 5, 7, 9, 11],  # Tester différentes sélections de variables
    "model__fit_intercept": [True, False] # Ajout de biais ou non 
}

# Recherche du RMSE minimum avec validation croisée 
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et scores optimisés
best_rmse = -grid_search.best_score_  #  Inversion car GridSearchCV retourne une valeur négative
best_r2 = grid_search.best_estimator_.score(X_test, y_test)  # Score R² du meilleur modèle

print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur RMSE en validation croisée : {best_rmse:.2f}")
print(f"R² du meilleur modèle : {best_r2:.3f}")

['id', 'date', 'prix', 'nb_chambres', 'nb_sdb', 'm2_interieur', 'm2_jardin', 'm2_etage', 'm2_soussol', 'nb_etages', 'vue_mer', 'vue_note', 'etat_note', 'design_note', 'annee_construction', 'annee_renovation', 'm2_interieur_15voisins', 'm2_jardin_15voisins', 'zipcode', 'lat', 'long', 'prix_moy_zipcode', 'year_sold', 'month_sold', 'day_of_week_sold', 'house_age', 'years_since_renovation', 'total_m2', 'mean_price_zipcode', 'm2_ratio', 'bathroom_per_bedroom', 'floor_utilization']
Number of columns in X: 17
RMSE du modèle : 194512.38
R² du modèle : 0.726
Meilleurs paramètres : {'feature_selection__k': 11, 'model__fit_intercept': True}
Meilleur RMSE en validation croisée : 178646.15
R² du meilleur modèle : 0.773
